In [1]:
#Import required libraries
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
# Helper: Early stopping.
early_stopper = EarlyStopping(patience=10)

Using TensorFlow backend.
W0615 16:41:26.569255 139705520310080 deprecation_wrapper.py:118] From /home/zander/anaconda3/lib/python3.7/site-packages/tensorflow/__init__.py:97: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0615 16:41:26.569831 139705520310080 deprecation_wrapper.py:118] From /home/zander/anaconda3/lib/python3.7/site-packages/tensorflow/__init__.py:97: The name tf.AttrValue is deprecated. Please use tf.compat.v1.AttrValue instead.

W0615 16:41:26.570591 139705520310080 deprecation_wrapper.py:118] From /home/zander/anaconda3/lib/python3.7/site-packages/tensorflow/__init__.py:97: The name tf.COMPILER_VERSION is deprecated. Please use tf.version.COMPILER_VERSION instead.

W0615 16:41:26.570943 139705520310080 deprecation_wrapper.py:118] From /home/zander/anaconda3/lib/python3.7/site-packages/tensorflow/__init__.py:97: The name tf.CXX11_ABI_FLAG is deprecated. Please use tf.sysconfig.CXX11_ABI_FLAG instead.

W0615 16:41:26.571360 13970552

In [2]:
#Import data saved from data cleaning
X_train = pd.read_csv('training_features.csv', sep='\t')
Y_train = pd.read_csv('training_target.csv', sep='\t')
#Observe that is a class imbalance 80-20, should be fine. 
#Y_train['y'].value_counts(normalize=True)
#Step 3 - Generate predictions:
#Create predictions on the data in test.csv using each of your trained models.  
#The predictions should be the class probabilities for belonging to the positive class (labeled '1').  
#Because of the above, I one hot encoded the targets and used softmax in the model
Y_train = pd.get_dummies(Y_train['y'])

In [17]:
def create_and_compile(network, input_shape, num_classes):
    """args is a dictionary"""
    nb_layers = network['nb_layers']
    nb_neurons = network['nb_neurons']
    activation = network['activation']
    optimizer = network['optimizer']
    #Define the type of model
    model = Sequential()
    model.add(Dense(nb_neurons, activation=activation, input_shape=input_shape, name='input_layer'))
    model.add(Dropout(0.2, name='dropout_input'))
    for i in range(nb_layers):
        model.add(Dense(nb_neurons, activation=activation, name='layer_{}'.format(i+1)))
        #Dropout to help avoid over-fitting
        model.add(Dropout(0.2, name='dropout_{}'.format(i+1)))
    # Output layer.
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])
    return model

def evaluate_model(network, X_train,Y_train):
    input_shape = (X_train.shape[1],)
    model = create_and_compile(network, input_shape,2)
    early_stopper = EarlyStopping(patience=10)
    model.fit(X_train, Y_train, verbose=0, epochs=100, validation_split=.25,callbacks=[early_stopper])
    acc = model.evaluate(X_train,Y_train)[1]
    return acc

In [18]:
nn_param_choices = {
        'nb_layers': [1, 2, 3, 4],
        'nb_neurons': [64, 128, 256, 512, 768, 1024],
        'activation': ['relu', 'elu', 'tanh', 'sigmoid'],
        'optimizer': ['rmsprop', 'adam', 'sgd', 'adagrad',
                      'adadelta', 'adamax', 'nadam']
    }
import itertools
keys, values = zip(*nn_param_choices.items())
experiments = [dict(zip(keys, v)) for v in itertools.product(*values)]

672

In [ ]:
new_list = []
for x in range(25):
    network = random.choice(experiments)
    print('Evaluating model number {}'.format(x+1))
    new_list.append(evaluate_model(network, X_train,Y_train))

In [21]:
#Define a helpful way to change out the values without messing with the overall flow
#I only tried a few combinations of neurons, layers and functions before I got to above 95% accuracy
network = {'nb_layers': 4, 'nb_neurons': 128, 'activation': 'elu', 'optimizer': 'rmsprop'}
nb_layers = network['nb_layers']
nb_neurons = network['nb_neurons']
activation = network['activation']
optimizer = network['optimizer']
input_shape = (X_train.shape[1],)
#Define the type of model
model = Sequential()
model.add(Dense(nb_neurons, activation=activation, input_shape=input_shape, name='input_layer'))
model.add(Dropout(0.2, name='dropout_input'))
for i in range(nb_layers):
    model.add(Dense(nb_neurons, activation=activation, name='layer_{}'.format(i+1)))
    #Dropout to help avoid over-fitting
    model.add(Dropout(0.2, name='dropout_{}'.format(i+1)))
# Output layer.
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=optimizer,
                  metrics=['accuracy'])

In [24]:
model.fit(X_train, Y_train, verbose=2, epochs=250, validation_split=.25,callbacks=[early_stopper])

Train on 30000 samples, validate on 10000 samples
Epoch 1/250
 - 3s - loss: 0.1471 - acc: 0.9815 - val_loss: 0.1280 - val_acc: 0.9681
Epoch 2/250
 - 6s - loss: 0.1394 - acc: 0.9821 - val_loss: 0.1327 - val_acc: 0.9815
Epoch 3/250
 - 12s - loss: 0.1256 - acc: 0.9838 - val_loss: 0.1029 - val_acc: 0.9866
Epoch 4/250
 - 12s - loss: 0.1245 - acc: 0.9840 - val_loss: 0.0813 - val_acc: 0.9857
Epoch 5/250
 - 12s - loss: 0.1211 - acc: 0.9825 - val_loss: 0.1067 - val_acc: 0.9839
Epoch 6/250
 - 12s - loss: 0.1134 - acc: 0.9850 - val_loss: 0.1202 - val_acc: 0.9889
Epoch 7/250
 - 12s - loss: 0.1230 - acc: 0.9838 - val_loss: 0.1069 - val_acc: 0.9848
Epoch 8/250
 - 12s - loss: 0.1170 - acc: 0.9840 - val_loss: 0.1036 - val_acc: 0.9882
Epoch 9/250
 - 12s - loss: 0.1221 - acc: 0.9849 - val_loss: 0.1126 - val_acc: 0.9880
Epoch 10/250
 - 12s - loss: 0.1189 - acc: 0.9855 - val_loss: 0.1146 - val_acc: 0.9861
Epoch 11/250
 - 12s - loss: 0.1132 - acc: 0.9845 - val_loss: 0.1090 - val_acc: 0.9885
Epoch 12/250
 -

In [25]:
model.evaluate(X_train,Y_train)[1]

40000/40000 [==============================] - 7s 185us/step


0.992125

In [26]:
#Save modeal for serving and re-loading
model.save('model_2.h5')

In [27]:
#Check accuracy
preds = model.predict(X_train)
X_test = pd.read_csv('testing_targets.csv', sep='\t')
preds = model.predict(X_test)
pd.DataFrame(preds).to_csv('results2.csv', index=False)